In [1]:
from tensorflow.keras.models import Sequential  

from tensorflow.keras.optimizers import SGD  

from tensorflow.keras.layers import Activation, Dense, Dropout, GRU  

import matplotlib.pyplot as plt  

import numpy as np  

import pandas as pd  

from sklearn.metrics import mean_absolute_error 

In [2]:
# 读取CSV文件中的数据  

data = pd.read_csv("BTC_DATA_V3.0.csv")  

# 选取除“Price”和“Vol.”列外的前六列  

data = data.iloc[:, 0:6]  

# 将“Price”列作为目标变量y  

y = data.loc[:, ['Price']]  

# 从原始数据中删除“Price”和“Vol.”列  

data = data.drop(['Price', 'Vol.'], axis='columns')  
 

# 打印处理后的数据的前五行  

print(data.head(5))  

# 打印目标变量y的前五行  

print(y.head(5))

        Date     Open     High      Low
0  25-Mar-22  44013.0  45112.0  43622.0
1  24-Mar-22  42911.0  44251.0  42658.0
2  23-Mar-22  42373.0  43027.0  41795.0
3  22-Mar-22  41022.0  43327.9  40893.0
4  21-Mar-22  41282.0  41532.0  40530.0
     Price
0  44331.0
1  44013.0
2  42912.0
3  42373.0
4  41022.0


In [3]:
data = data.set_index('Date')  # 将'Date'列设置为数据框的索引  

data.index = pd.to_datetime(data.index, unit='ns')  # 将索引转换为datetime64[ns]类型  

print(data.index)  # 打印转换后的日期索引

DatetimeIndex(['2022-03-25', '2022-03-24', '2022-03-23', '2022-03-22',
               '2022-03-21', '2022-03-20', '2022-03-19', '2022-03-18',
               '2022-03-17', '2022-03-16',
               ...
               '2018-03-07', '2018-03-06', '2018-03-05', '2018-03-04',
               '2018-03-03', '2018-03-02', '2018-03-01', '2018-02-28',
               '2018-02-27', '2018-02-26'],
              dtype='datetime64[ns]', name='Date', length=1489, freq=None)


In [4]:
aim = 'Price'  # 设定目标变量为'Price'

In [5]:
data.shape  # 查询数据框的形状

(1489, 3)

In [6]:
X_train = data[300:]  # 将数据框从第300行开始到末尾作为训练集特征  

X_test = data[:300]   # 将数据框从开始到第300行（不包括第300行）作为测试集特征  

y_train = y[300:]     # 将目标变量从第300行开始到末尾作为训练集标签  

y_test = y[:300]      # 将目标变量从开始到第300行（不包括第300行）作为测试集标签  

print(y_test)         # 打印测试集标签

       Price
0    44331.0
1    44013.0
2    42912.0
3    42373.0
4    41022.0
..       ...
295  39187.3
296  37555.8
297  36687.6
298  37298.6
299  35652.8

[300 rows x 1 columns]


In [7]:
def normalise_zero_base(continuous): return continuous / continuous.iloc[0] - 1  

def normalise_min_max(continuous): return (continuous - continuous.min()) / (data.max() - continuous.min())
# normalise_zero_base 函数将连续数据以第一行数据为基准进行归一化，即除以第一行数据然后减去1。
#normalise_min_max 函数将连续数据缩放到0到1之间，通过减去最小值并除以数据范围（最大值减去最小值）实现。但注意这里使用了全局的 data.max() 而不是 continuous.max()，这可能是一个错误，因为通常我们会希望针对 continuous 自身的最大值和最小值进行归一化。

In [8]:
X_train = normalise_zero_base(X_train)  

X_test = normalise_zero_base(X_test)  

y_train = normalise_zero_base(y_train)  

y_test = normalise_zero_base(y_test)

In [9]:
import numpy as np  

X_train = np.expand_dims(X_train, axis=1)  

X_test = np.expand_dims(X_test, axis=1)

In [27]:
from tensorflow import keras  # 从 TensorFlow 导入 Keras 模块  

from tensorflow.keras.layers import GRU, Dropout, PReLU
# 定义 GRU 模型  

gruMODEL = keras.Sequential()  # 创建一个顺序模型  

  

# 添加第一个 GRU 层，并应用了 Dropout 正则化  

gruMODEL.add(keras.layers.GRU(  

    units=1024,  # 隐藏层单元数为 1024  

    input_shape=(1, 3),  # 输入形状为 (时间步长, 特征数)，这里时间步长为 1，特征数为 3  

    #activation='PReLU',  # 激活函数为 PReLU（参数化修正线性单元） 
    activation = "relu",

    recurrent_activation="sigmoid",  # 循环层激活函数为 sigmoid  

    use_bias=True,  # 使用偏置项  

    kernel_initializer="glorot_uniform",  # 权重初始化方法为 Glorot 均匀分布  

    recurrent_initializer="orthogonal",  # 循环层权重初始化方法为正交矩阵  

    bias_initializer="zeros",  # 偏置项初始化方法为 0  
    # 以下均为正则化和约束选项，此处未使用  

    kernel_regularizer=None,  

    recurrent_regularizer=None,  

    bias_regularizer=None,  

    activity_regularizer=None,  

    kernel_constraint=None,  

    recurrent_constraint=None,  

    bias_constraint=None,  

    dropout=0.0,  # 在输入层上丢弃的输入单元比例，此处未使用  

    recurrent_dropout=0.0,  # 在循环层上丢弃的单元比例，此处未使用  

    return_sequences=False,  # 是否返回完整序列的输出，这里只返回最后一个时间步的输出  

    return_state=False,  # 是否返回最后一个时间步的状态，此处不返回  

    go_backwards=False,  # 是否反向处理输入序列  

    stateful=False,  # 如果为 True，则批次中的每个样本的索引 i 的状态将被用作下一个批次中索引 i 的样本的初始状态  

    unroll=False,  # 如果为 True，则网络将展开，否则使用符号循环  

    # time_major=False,  # 输入和输出的形状格式，False 表示 (batch, time, features)，True 表示 (time, batch, features)  

    reset_after=True  # 是否在每个时间步重置门控单元  

))  

  

# 添加 Dropout 层，丢弃率为 0.9 
# 添加 PReLU 激活函数作为独立层
gruMODEL.add(PReLU())

gruMODEL.add(keras.layers.Dropout(0.9))
gruMODEL.compile(optimizer="adam", loss="mean_squared_error")


C:\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
gruMODEL.fit(X_train, y_train)

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.5346


In [37]:
# 进行预测
# predictions = gruMODEL.predict(X_test)
predictions = gruMODEL.inverse_transform(gruMODEL.predict(X_test))
# 打印预测结果
print(predictions)


AttributeError: 'Sequential' object has no attribute 'inverse_transform'

In [33]:
predictions.shape

(300, 1024)

In [31]:
y_test

,Price
0,0.000000
1,-0.007173
2,-0.032009
3,-0.044168
4,-0.074643
...,...
295,-0.116029
296,-0.152832
297,-0.172417
298,-0.158634


In [20]:
from tensorflow import keras  

from tensorflow.keras.models import Sequential  

from tensorflow.keras.layers import GRU, Dropout, PReLU

  

# 初始化 GRU 模型  

gruMODEL = Sequential()  

  

# 第一层 GRU，并添加 Dropout 正则化  

gruMODEL.add(GRU(  

    units=1024,  # 隐藏单元数量  

    input_shape=(1, 3),  # 输入数据的形状  

    # activation='PReLU',  # 激活函数  
    activation = "relu",

    recurrent_activation="sigmoid",  # 循环层激活函数  

    use_bias=True,  # 是否使用偏置项  

    kernel_initializer="glorot_uniform",  # 权重初始化方法  

    recurrent_initializer="orthogonal",  # 循环权重初始化方法  

    bias_initializer="zeros",  # 偏置项初始化方法  

    # 正则化、约束等参数均设置为 None，即不使用  

    dropout=0.0,  # 在每个时间步的输入之间丢弃一部分单元  

    recurrent_dropout=0.0,  # 在每个时间步的循环层之间丢弃一部分单元  

    return_sequences=False,  # 是否返回输出序列的最后一个时间步  

    return_state=False,  # 是否返回最后一个时间步的状态  

    go_backwards=False,  # 是否反向处理输入序列  

    stateful=False,  # 如果为 True，则批次中的每个样本将具有其自己的内部状态  

    unroll=False,  # 如果为 True，则网络将展开其计算图，这可能会消耗更多的内存但会加速计算  

    #time_major=False,  # 输入数据的形状格式，此处为（batch, time, features）  

    reset_after=True  # 在计算输出时，是否重置隐藏状态  

))  
# 添加 PReLU 激活函数作为独立层
# gruMODEL.add(PReLU())

gruMODEL.add(Dropout(0.9))  # 添加 Dropout 层，丢弃率为 0.9  



# 第二层 GRU  

gruMODEL.add(GRU(  

    units=2048,  # 隐藏单元数量  

    # activation='PReLU',  # 激活函数  
    activation = "relu",

    recurrent_activation="sigmoid",  # 循环层激活函数  

    use_bias=True,  # 是否使用偏置项  

    kernel_initializer="glorot_uniform",  # 权重初始化方法  

    recurrent_initializer="orthogonal",  # 循环权重初始化方法  

    bias_initializer="zeros",  # 偏置项初始化方法  

    # 正则化、约束等参数均设置为 None，即不使用  

    dropout=0.0,  # 在每个时间步的输入之间丢弃一部分单元  

    recurrent_dropout=0.0,  # 在每个时间步的循环层之间丢弃一部分单元  

    return_sequences=False,  # 是否返回输出序列的最后一个时间步  

    return_state=False,  # 是否返回最后一个时间步的状态  

    go_backwards=False,  # 是否反向处理输入序列  

    stateful=False,  # 如果为 True，则批次中的每个样本将具有其自己的内部状态  

    unroll=False,  # 如果为 True，则网络将展开其计算图  

    # time_major=False,  # 输入数据的形状格式  

    reset_after=True  # 在计算输出时，是否重置隐藏状态  

))  
# 添加 PReLU 激活函数作为独立层
# gruMODEL.add(PReLU())

gruMODEL.add(Dropout(0.8))  # 添加 Dropout 层，丢弃率为 0.8
# 在这个模型中，我们首先定义了一个 Sequential 模型，然后添加了两个 GRU 层，每个 GRU 层后面都跟随了一个 Dropout 层。第一层 GRU 的隐藏单元数为 1024，输入数据的形状为 (1, 3)，表示每个样本有一个时间步长和三个特征。第二层 GRU 的隐藏单元数增加到了 2048，并且两个 Dropout 层的丢弃率分别设置为 0.9 和 0.8，用于在训练过程中随机丢弃一部分神经元的输出，以减少过拟合的风险。

# 第三层GRU层

gruMODEL.add(GRU(  
    units=4096, # 神经元数量  
    # activation='PReLU', # 激活函数为PReLU  
    activation = "relu",
    recurrent_activation="sigmoid", # 递归激活函数为sigmoid  
    use_bias=True, # 使用偏置项  
    kernel_initializer="glorot_uniform", # 权重初始化方法  
    recurrent_initializer="orthogonal", # 递归权重初始化方法  
    bias_initializer="zeros", # 偏置初始化方法  
    kernel_regularizer=None, # 权重正则化方法  
    recurrent_regularizer=None, # 递归权重正则化方法  
    bias_regularizer=None, # 偏置正则化方法  
    activity_regularizer=None, # 激活正则化方法  
    kernel_constraint=None, # 权重约束  
    recurrent_constraint=None, # 递归权重约束  
    bias_constraint=None, # 偏置约束  
    dropout=0.0, # dropout层比例  
    recurrent_dropout=0.0, # 递归dropout层比例  
    return_sequences=False, # 是否返回输出序列的最后一个输出  
    return_state=False, # 是否返回最后一个状态  
    go_backwards=False, # 是否反向计算  
    stateful=False, # 是否是有状态的网络  
    unroll=False, # 是否展开网络  
    # time_major=False, # 输入张量的形状格式  
    reset_after=True)) # 是否在每个时间步之后重置门控单元
# 添加 PReLU 激活函数作为独立层
# gruMODEL.add(PReLU())
gruMODEL.add(Dropout(0.09)) # 添加dropout层，防止过拟合

# 第四层GRU层

gruMODEL.add(GRU(  
    units=512, # 神经元数量  
    # activation='PReLU', # 激活函数为PReLU  
    activation = "relu",
    recurrent_activation="sigmoid", # 递归激活函数为sigmoid  
    use_bias=True, # 使用偏置项  
    kernel_initializer="glorot_uniform", # 权重初始化方法  
    recurrent_initializer="orthogonal", # 递归权重初始化方法  
    bias_initializer="zeros", # 偏置初始化方法  
    kernel_regularizer=None, # 权重正则化方法  
    recurrent_regularizer=None, # 递归权重正则化方法  
    bias_regularizer=None, # 偏置正则化方法  
    activity_regularizer=None, # 激活正则化方法  
    kernel_constraint=None, # 权重约束  
    recurrent_constraint=None, # 递归权重约束  
    bias_constraint=None, # 偏置约束  
    dropout=0.0, # dropout层比例  
    recurrent_dropout=0.0, # 递归dropout层比例  
    return_sequences=False, # 是否返回输出序列的最后一个输出  
    return_state=False, # 是否返回最后一个状态  
    go_backwards=False, # 是否反向计算  
    stateful=False, # 是否是有状态的网络  
    unroll=False, # 是否展开网络  
    # time_major=False, # 输入张量的形状格式  
    reset_after=True)) # 是否在每个时间步之后重置门控单元

gruMODEL.add(Dropout(0.09)) # 添加dropout层，防止过拟合

# 输出层

gruMODEL.add(Dense(units=1)) # 添加全连接层，输出单元数为1

# 编译RNN

gruMODEL.compile(optimizer="sgd", # 使用随机梯度下降优化器  
loss='mean_squared_error') # 使用均方误差作为损失函数  
gruMODEL.summary() # 打印模型概览信息

ValueError: Input 0 of layer "gru_8" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 1024)

In [ ]:
# 定义网络层数和每层神经元数量  

layers = [3,15,30,45,90,1]  

# 定义各层的文本标签  

layers_str = ["Input"] + ["GRU"] * (len(layers) - 2) + ["Output"]  

# 定义各层的颜色属性（这里实际未使用）  

layers_col = ["none"] + ["none"] * (len(layers) - 2) + ["none"]  

# 定义各层的填充颜色  

layers_fill = ["black"] + ["gray"] * (len(layers) - 2) + ["black"]  

# 节点边框宽度  

penwidth = 15  

# 字体设置  

font = "Hilda 10"  
  

# 开始打印DOT语言描述  

print("digraph G {")  

print("\tfontname = "{}"".format(font))  

print("\trankdir=LR")  # 设置图的布局方向为从左到右  

print("\tsplines=line")  # 设置边为直线  

print("\tnodesep=.08;")  # 设置节点之间的间距  

print("\tranksep=1;")  # 设置层次之间的间距  

print("\tedge [color=black, arrowsize=.5];")  # 设置边的颜色和箭头大小  

print("\tnode [fixedsize=true,label="",style=filled," + \  

      "color=none,fillcolor=gray,shape=circle]\n")  # 设置节点属性，如固定大小、填充样式和颜色等  

  

# 绘制各层（子图）  

for i in range(0, len(layers)):  

    print("\tsubgraph cluster_{} {{".format(i))  

    print("\t\tcolor={};".format(layers_col[i]))  # 设置子图颜色（这里实际未使用）  

    print("\t\tnode [style=filled, color=white, penwidth={},"  

          "fillcolor={} shape=circle];".format(penwidth, layers_fill[i]))  # 设置子图中节点的属性  

    print("\t\t", end=' ')  

    for a in range(layers[i]):  

        print("l{}{} ".format(i + 1, a), end=' ')  # 打印节点标签  

    print(";")  

    print("\t\tlabel = {};".format(layers_str[i]))  # 设置子图的标签  

    print("\t}\n")  


## 绘制节点间的边  

for i in range(1, len(layers)):  

    for a in range(layers[i - 1]):  

        for b in range(layers[i]):  

            print("\tl{}{} -> l{}{}".format(i, a, i + 1, b))  # 绘制从第i层的第a个节点到第i+1层的第b个节点的边  

print("}")  # 结束DOT语言描述

In [ ]:
# 使用'w'模式打开名为'model.txt'的文件，准备写入网络层结构信息  

with open('model.txt', 'w') as layers_file:  

    # 定义网络各层的神经元数量  

    layers = [3,5,10,15,20,1]  

    # 定义各层的文本标签  

    layers_str = ["Input"] + ["GRU"] * (len(layers) - 2) + ["Output"]  

    # 定义各层的颜色属性（此处未实际使用）  

    layers_col = ["none"] + ["none"] * (len(layers) - 2) + ["none"]  

    # 定义各层的填充颜色  

    layers_fill = ["black"] + ["gray"] * (len(layers) - 2) + ["black"]  

    # 节点边框宽度  

    penwidth = 15  

    # 字体设置  

    font = "Hilda 10"  

  

    # 写入DOT语言描述，用于描述网络结构图  

    layers_file.write("digraph G {\n")  # 开始定义有向图G  

    layers_file.write("\tfontname = "{}"\n".format(font))  # 设置字体名称为Hilda 10  

    layers_file.write("\trankdir=LR\n")  # 设置图的布局方向为从左到右  

    layers_file.write("\tsplines=line\n")  # 设置边为直线  

    layers_file.write("\tnodesep=.08;\n")  # 设置节点之间的间距  

    layers_file.write("\tranksep=1;\n")  # 设置层次之间的间距  

    layers_file.write("\tedge [color=black, arrowsize=.5];\n")  # 设置边的颜色和箭头大小  

    layers_file.write("\tnode [fixedsize=true,label="",style=filled,color=none,fillcolor=gray,shape=circle]\n\n")  # 设置节点属性  

  

    # 绘制各层（子图）  

    for i in range(0, len(layers)):  

        layers_file.write("\tsubgraph cluster_{} {{\n".format(i))  # 开始定义第i个子图  

        layers_file.write("\t\tcolor={};\n".format(layers_col[i]))  # 设置子图颜色（此处未实际使用）  

        layers_file.write("\t\tnode [style=filled, color=white, penwidth={},fillcolor={} shape=circle];\n".format(penwidth, layers_fill[i]))  # 设置子图中节点的属性  

        layers_file.write("\t\t")  

        for a in range(layers[i]):  

            layers_file.write("l{}{} ".format(i + 1, a))  # 写入节点标签  

        layers_file.write(";\n")  

        layers_file.write("\t\tlabel = {};\n".format(layers_str[i]))  # 设置子图的标签  

        layers_file.write("\t}\n\n")  # 结束定义第i个子图  

  

    # 绘制节点间的边  

    for i in range(1, len(layers)):  

        for a in range(layers[i - 1]):  

            for b in range(layers[i]):  

                layers_file.write("\tl{}{} -> l{}{}\n".format(i, a, i + 1, b))  # 绘制从第i层的第a个节点到第i+1层的第b个节点的边  

  

    layers_file.write("}\n")  # 结束DOT语言描述

In [ ]:
from sklearn.metrics import r2_score  

r2_score = r2_score(y_test, preds)  

r2_score * 100

In [ ]:
prediction = np.array([[44331,44818,44090]])  # 这里假设的预测数据，用于后续计算  

X_testt = scaling.inverse_transform(X_test[0])  # 对测试数据进行逆缩放处理  

prediction_new = np.array([[(X_test[0][0]/X_testt[0]*prediction[0])]])  # 基于缩放后的数据重新计算预测值  

predictions = gruMODEL.predict(prediction_new)[0][0]  # 使用GRU模型进行预测  

predictions = np.array([[predictions]]) * prediction[0][0] / prediction_new[0][0][0]  # 对预测结果进行比例调整  

f"27 March Prediction is {predictions[0][0]} BTC/USDT"  # 输出预测结果